In [1]:
import gurobipy as gp


In [2]:
V = [1, 2, 3, 4, 5, 6, 7, 8, 9]
V_0 = [0] + V
n = len(V)  # numero de clientes
Q = 500  # capacidade
MD = 7000  # Distancia Maxima para todo k
K = 3  # numero de veiculos disponiveis


In [3]:
Distance_Matrix = [[0, 300, 713, 890, 828, 978, 1202, 476, 974, 1253],
                   [300, 0, 825, 721, 990, 1031, 1048, 281, 1029, 1092],
                   [713, 825, 0, 746, 1468, 1681, 903, 689, 1676, 961],
                   [890, 721, 746, 0, 1696, 1749, 328, 441, 1747, 371],
                   [828, 990, 1467, 1696, 0, 385, 2018, 1257, 379, 2066],
                   [978, 1031, 1680, 1749, 385, 0, 2077, 1313, 6, 2118],
                   [1202, 1048, 903, 328, 2018, 2077, 0, 768, 2075, 59],
                   [476, 281, 689, 441, 1257, 1313, 768, 0, 1310, 812],
                   [974, 1029, 1676, 1747, 379, 6, 2075, 1310, 0, 2116],
                   [1253, 1092, 961, 371, 2066, 2118, 59, 812, 2116, 0]]
Demandas = {}
Demandas['coleta'] = [0, 62, 17, 50, 6, 49, 9, 9, 15, 51]
Demandas['entregas'] = [0, 44, 55, 38, 56, 28, 8, 28, 42, 32]
veiculos = [i + 1 for i in range(K)]


In [4]:
m = gp.Model("VRPSPD")


In [5]:
# variaveis
x = m.addVars(V_0, V_0, veiculos, vtype=gp.GRB.BINARY, name="x")
y = m.addVars(V_0, V_0, vtype=gp.GRB.INTEGER, name="y")  # coleta
z = m.addVars(V_0, V_0, vtype=gp.GRB.INTEGER, name="z")  # entrega


In [6]:
obj = gp.LinExpr()
for k in veiculos:
    for i in V_0:
        for j in V_0:
            obj += Distance_Matrix[i][j] * x[i, j, k]


In [7]:
# funcao objetivo
m.setObjective(obj, sense=gp.GRB.MINIMIZE)


In [8]:
# restrições
r1 = m.addConstrs(
    (gp.quicksum(x[i, j, k] for i in V_0 for k in veiculos) == 1 for j in V), name="X_ijk_eq_1"
)
r2 = m.addConstrs(
    (
        (gp.quicksum(x[i, j, k] for i in V_0)) -
        (gp.quicksum(x[j, i, k] for i in V_0)) == 0
        for j in V_0
        for k in veiculos
    ), name="chega_e_sai"
)
r3 = m.addConstrs(
    (
        gp.quicksum(x[0, j, k] for j in V) <= 1
        for k in veiculos
    ),
    name="max_k_veiculos"
)
r4 = m.addConstrs(
    (
        gp.quicksum(Distance_Matrix[i][j]*x[i, j, k]
                    for i in V_0 for j in V_0) <= MD
        for k in veiculos
    ),
    name="distancia_max_k"
)
r5 = m.addConstrs(
    (
        gp.quicksum(y[j, i] for i in V_0) - gp.quicksum(y[i, j] for i in V_0)
        == Demandas['coleta'][j] for j in V_0 if j != 0
    ),
    name="fluxo_y"
)
r6 = m.addConstrs(
    (
        gp.quicksum(z[i, j] for i in V_0) - gp.quicksum(z[j, i] for i in V_0)
        == Demandas['entregas'][j] for j in V_0 if j != 0
    ),
    name="fluxo_z"
)
r7 = m.addConstrs(
    (
        y[i, j] + z[i, j] <= Q*gp.quicksum(x[i, j, k] for k in veiculos)
        for i in V_0
        for j in V_0
    ),
    name="capacidade"
)


In [9]:
m.update()


In [10]:
m.write("modelo1.lp")


In [11]:
m.optimize()


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-3550 CPU @ 3.30GHz, instruction set [SSE2|AVX]
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 163 rows, 500 columns and 1931 nonzeros
Model fingerprint: 0xddf7a501
Variable types: 0 continuous, 500 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [6e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
Presolve removed 10 rows and 50 columns
Presolve time: 0.01s
Presolved: 153 rows, 450 columns, 1830 nonzeros
Variable types: 0 continuous, 450 integer (270 binary)

Root relaxation: objective 3.358356e+03, 217 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3358.35600    0   26          - 3358.35600      -     -    0s

In [13]:
m.write("formulacao.txt", "LP")


TypeError: write() takes exactly 2 positional arguments (3 given)